In [1]:
!nvidia-smi

Fri Apr  9 15:36:57 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P0    74W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
pip install regex requests

In [4]:
import functools
import importlib
from copy import deepcopy

In [5]:
 pip install hydra-core --upgrade

Requirement already up-to-date: hydra-core in /usr/local/lib/python3.7/dist-packages (1.0.6)


In [6]:
pip install datasets

In [7]:
pip install allennlp

In [8]:
pip install transformers

In [9]:
import functools
import importlib

dependencies = [
    "dataclasses",
    "hydra",
    "numpy",
    "omegaconf",
    "regex",
    "requests",
    "torch",
]

missing_deps = []
for dep in dependencies:
    try:
        importlib.import_module(dep)
    except ImportError:
        if dep == "hydra":
            dep = "hydra-core"
        missing_deps.append(dep)

In [10]:
import torch

In [11]:
roberta = torch.hub.load('pytorch/fairseq', 'roberta.large.mnli')
roberta.eval()

Using cache found in /root/.cache/torch/hub/pytorch_fairseq_master


RobertaHubInterface(
  (model): RobertaModel(
    (encoder): RobertaEncoder(
      (sentence_encoder): TransformerEncoder(
        (dropout_module): FairseqDropout()
        (embed_tokens): Embedding(50265, 1024, padding_idx=1)
        (embed_positions): LearnedPositionalEmbedding(514, 1024, padding_idx=1)
        (layernorm_embedding): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (layers): ModuleList(
          (0): TransformerEncoderLayer(
            (self_attn): MultiheadAttention(
              (dropout_module): FairseqDropout()
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (dropou

In [12]:
from datasets import load_dataset, load_metric
from allennlp.common.util import lazy_groups_of
import numpy as np
from fairseq.data.data_utils import collate_tokens
import heapq
from operator import itemgetter

In [13]:
actual_task = "mnli"
dataset = load_dataset("glue", actual_task)
metric = load_metric('glue', actual_task)

Reusing dataset glue (/root/.cache/huggingface/datasets/glue/mnli/1.0.0/7c99657241149a24692c402a5c3f34d4c9f1df5ac2e4c3759fadea38f6cb29c4)


In [14]:
from transformers import AutoModel, AutoTokenizer

tokenizer1 = AutoTokenizer.from_pretrained("roberta-large-mnli")

In [15]:
extracted_grads = []
def extract_grad_hook(module, grad_in, grad_out):
    extracted_grads.append(grad_out[0])

In [16]:
def add_hooks(language_model):
    for module in language_model.modules():
        if isinstance(module, torch.nn.Embedding):
            print(module.weight.shape[0])
            if module.weight.shape[0] == 50265:
                module.weight.requires_grad = True
                module.register_backward_hook(extract_grad_hook)

In [17]:
add_hooks(roberta)

50265
514


In [18]:
def get_embedding_weight(language_model):
    for module in language_model.modules():
        if isinstance(module, torch.nn.Embedding):
            if module.weight.shape[0] == 50265:
                return module.weight.detach()

In [19]:
embedding_weight = get_embedding_weight(roberta)

In [20]:
dataset_label_filter = 0

In [21]:
#в датасете за логическое следствие отвечает 0, у модели - 2, поэтому поменяем 0 на 2
#сохраняем примерно 5% от всех логических следствий в датасете
subset_dev_dataset = []
for instance in dataset['train']:
    if instance['label'] == dataset_label_filter:
        if np.random.rand() < 0.05:
          d = {'hypothesis':instance['hypothesis'], 'label':2-instance['label'], 'premise':instance['premise']}
          subset_dev_dataset.append(d)

In [22]:
target_label = 0

In [23]:
len(subset_dev_dataset)

6610

In [24]:
from torch.utils.data import Dataset, DataLoader
from scipy.special import softmax

In [25]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [26]:
roberta.to(device)

RobertaHubInterface(
  (model): RobertaModel(
    (encoder): RobertaEncoder(
      (sentence_encoder): TransformerEncoder(
        (dropout_module): FairseqDropout()
        (embed_tokens): Embedding(50265, 1024, padding_idx=1)
        (embed_positions): LearnedPositionalEmbedding(514, 1024, padding_idx=1)
        (layernorm_embedding): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (layers): ModuleList(
          (0): TransformerEncoderLayer(
            (self_attn): MultiheadAttention(
              (dropout_module): FairseqDropout()
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (dropou

In [27]:
#для оценки лосса на подменяемых таргетах (те на противоречия)
def evaluate_batch_on_target(model, batch, trigger_token=None, loss=False):
    res_batch = []
    for i in range(len(batch[0]['premise']) // 8):
      d = {'hypothesis':batch[0]['hypothesis'][8*i:min(8*(i+1), len(batch[0]['hypothesis']))],
           'label':[target_label for i in range(min(8*(i+1), len(batch[0]['label'])) - 8*i)],
           'premise':batch[0]['premise'][8*i:min(8*(i+1), len(batch[0]['premise']))]}
      res_batch.append(d)
    length = len(batch[0]['premise'])
    del batch


    right = 0
    loss = 0
    for j in range(len(res_batch)):
      sentences = []
      for i in range(8):
        s = ''
        for k in trigger_token:
          s += k
        s += ' '
        sentences.append([s + res_batch[j]['premise'][i], res_batch[j]['hypothesis'][i]])
      sentences = collate_tokens([roberta.encode(sent1, sent2) for sent1, sent2 in sentences], pad_idx=1)
      tmp = roberta.predict('mnli', sentences).detach().cpu()
      pred = tmp.argmax(axis=1)
      for i in range(len(pred)):
          right += (pred[i] == target_label)
      m = torch.nn.Softmax()
      for i in range(len(pred)):
          now = m(tmp[i])
          loss -= torch.log(now[target_label])
    loss = loss / length
    return right.detach().cpu().numpy(), loss

In [28]:
def evaluate_batch(model, batch, trigger_token=None, loss=False):
    if trigger_token is None:
        right = 0
        sentences = []
        for i in range(len(batch[0]['premise'])):
          sentences.append([batch[0]['premise'][i], batch[0]['hypothesis'][i]])
        sentences = collate_tokens([roberta.encode(sent1, sent2) for sent1, sent2 in sentences], pad_idx=1)
        pred = roberta.predict('mnli', sentences).argmax(axis=1)
        for i in range(len(pred)):
            right += (pred[i] == batch[0]['label'][i])
        return right.detach().cpu().numpy(), _
    else:
        right = 0
        sentences = []
        for i in range(len(batch[0]['premise'])):
          s = ''
          for k in trigger_token:
            s += k
          s += ' '
          sentences.append([s + batch[0]['premise'][i], batch[0]['hypothesis'][i]])
        sentences = collate_tokens([roberta.encode(sent1, sent2) for sent1, sent2 in sentences], pad_idx=1)
        tmp = roberta.predict('mnli', sentences)
        pred = tmp.argmax(axis=1)
        for i in range(len(pred)):
            right += (pred[i] == batch[0]['label'][i])
        if loss:
            loss = 0
            m = torch.nn.Softmax()
            for i in range(len(pred)):
                now = m(tmp[i])
                loss -= torch.log(now[batch[0]['label'][i]])
            loss = loss / len(pred)
            return right.detach().cpu().numpy(), loss
        return right.detach().cpu().numpy(), _

In [29]:
def get_accuracy(model, dev_dataset, trigger_token=None, loss=False):
  if trigger_token is None:
      res = 0
      for batch in lazy_groups_of(DataLoader(dev_dataset, batch_size=8, shuffle=False), group_size=1):
          res += evaluate_batch(model, batch, trigger_token, loss)[0]
      print("Without Triggers: " + str(res / len(dev_dataset)))
  else:
      res = 0
      for batch in lazy_groups_of(DataLoader(dev_dataset, batch_size=8, shuffle=False), group_size=1):
          res += evaluate_batch(model, batch, trigger_token, loss)[0]
      s = ''
      for k in trigger_token:
        s += k + ' '
      print("Current Triggers: " + s + " : " + str(res / len(dev_dataset)))

In [30]:
def get_average_grad(model, batch, trigger_token, target_label=None):
    optimizer = torch.optim.Adam(model.parameters())
    optimizer.zero_grad()

    original_labels = batch[0]['label'].clone()

    if target_label is not None:
        batch[0]['label'] = int(target_label) * torch.ones_like(batch[0]['label']).cuda()

    global extracted_grads
    res_batch = []
    for i in range(len(batch[0]['premise']) // 8):
      d = {'hypothesis':batch[0]['hypothesis'][8*i:min(8*(i+1), len(batch[0]['hypothesis']))],
           'label':batch[0]['label'][8*i:min(8*(i+1), len(batch[0]['label']))],
           'premise':batch[0]['premise'][8*i:min(8*(i+1), len(batch[0]['premise']))]}
      res_batch.append(d)
    
    averaged_grad = None
    for i in res_batch:
        optimizer.zero_grad()
        extracted_grads = [] 
        loss = evaluate_batch(model, [i], trigger_token, True)[1]
        loss.backward()

        grads = extracted_grads[0].cpu()
        if averaged_grad is None:
          averaged_grad = torch.sum(grads, dim=0)[:4]
        else:
          averaged_grad += torch.sum(grads, dim=0)[:4]
    batch[0]['label'] = original_labels 
    
    return averaged_grad

In [31]:
def hotflip_attack(averaged_grad, embedding_matrix, trigger_token, num_candidates=1):
    averaged_grad = averaged_grad.cpu()
    embedding_matrix = embedding_matrix.cpu()

    averaged_grad = averaged_grad.unsqueeze(0)
    gradient_dot_embedding_matrix = torch.einsum("bij,kj->bik",
                                                 (averaged_grad, embedding_matrix))            
    
    gradient_dot_embedding_matrix *= -1  

    _, best_k_ids = torch.topk(gradient_dot_embedding_matrix, num_candidates, dim=2)
    return best_k_ids.detach().cpu().numpy()[0]

In [32]:
def get_loss_per_candidate(index, model, batch, trigger_token, cand_trigger_token):
    loss_per_candidate = []
    curr_loss = evaluate_batch_on_target(model, batch, trigger_token, True)[1].cpu().detach().numpy()
    loss_per_candidate.append((deepcopy(trigger_token), curr_loss))
    for cand_id in range(len(cand_trigger_token[index])):
        trigger_token_one_replaced = deepcopy(trigger_token) 
        trigger_token_one_replaced[index] = cand_trigger_token[index][cand_id]
        loss = evaluate_batch_on_target(model, batch, trigger_token_one_replaced, True)[1].cpu().detach().numpy()
        loss_per_candidate.append((deepcopy(trigger_token_one_replaced), loss))
    return loss_per_candidate

In [33]:
def get_best_candidates(model, batch, trigger_token, cand_trigger_token, beam_size=1):
    loss_per_candidate = get_loss_per_candidate(0, model, batch, trigger_token,
                                                cand_trigger_token)

    top_candidates = heapq.nsmallest(beam_size, loss_per_candidate, key=itemgetter(1))

    for idx in range(1, len(trigger_token)):
        loss_per_candidate = []
        for cand, _ in top_candidates:
            loss_per_candidate.extend(get_loss_per_candidate(idx, model, batch, cand,
                                                             cand_trigger_token))
        top_candidates = heapq.nsmallest(beam_size, loss_per_candidate, key=itemgetter(1))
    return min(top_candidates, key=itemgetter(1))[0]

In [34]:
get_accuracy(roberta, subset_dev_dataset)

Without Triggers: 0.9792738275340394


In [35]:
torch.cuda.empty_cache()

In [36]:
trigger_token = ['Not', ' means', ' opposite', ' the'] 
for epoch in range(1):
    for batch in lazy_groups_of(DataLoader(subset_dev_dataset, batch_size=len(subset_dev_dataset)//12, shuffle=True), group_size=1):
        get_accuracy(roberta, subset_dev_dataset, trigger_token)
        
        roberta.train() 
        torch.cuda.empty_cache()
        

        averaged_grad = get_average_grad(roberta, batch, trigger_token, target_label)

        cand_trigger_token_ids = hotflip_attack(averaged_grad,
                                                        embedding_weight,
                                                        trigger_token,
                                                        num_candidates=3)
        torch.cuda.empty_cache()

        cand_trigger_token = []
        for k in range(4):
          tmp = []
          for j in range(len(cand_trigger_token_ids[k])):
            tmp.append(tokenizer1.decode(int(cand_trigger_token_ids[k][j])))
          cand_trigger_token.append(tmp)

          trigger_token[k] = cand_trigger_token[k][0]

          print("Претенденты на место ", k, ": ", cand_trigger_token[k])
        

        trigger_token = get_best_candidates(roberta,
                                                  batch,
                                                  trigger_token,
                                                  cand_trigger_token)

        torch.cuda.empty_cache()

Current Triggers: Not  means  opposite  the  : 0.9747352496217851
Претенденты на место  0 :  ['aul', ' Tul', 'thia']
Претенденты на место  1 :  ['Answer', 'BUT', 'Used']
Претенденты на место  2 :  [' signifies', 'asca', ' �']
Претенденты на место  3 :  [' mapping', ' marrying', ' dressing']
Current Triggers: thia BUT  �  marrying  : 0.9633888048411497
Претенденты на место  0 :  [' Derek', ' arts', ' Shaw']
Претенденты на место  1 :  ['emate', ' Choi', ' Gerrard']
Претенденты на место  2 :  ['hirt', 'URN', ' dissolve']
Претенденты на место  3 :  ['iture', 'ilst', 'ucci']
Current Triggers:  Shaw  Gerrard hirt ilst  : 0.9635400907715582
Претенденты на место  0 :  ['ama', 'ocy', ' Rodgers']
Претенденты на место  1 :  [' BB', ' Kak', ' ONLY']
Претенденты на место  2 :  ['seys', ' Beckham', 'Football']
Претенденты на место  3 :  ['�', '�', ' wink']
Current Triggers: ocy  ONLY seys �  : 0.9649016641452345
Претенденты на место  0 :  ['imation', ' reactor', ' Molecular']
Претенденты на место  1

In [37]:
get_accuracy(roberta, subset_dev_dataset, trigger_token)

Current Triggers:  guardian Daddy squ  hindsight  : 0.967624810892587
